In [1]:
from datetime import date, datetime, timedelta

import pandas as pd

In [47]:
df = pd.DataFrame(
    {"date": pd.date_range(start="2024-02-01", end="2024-03-31", freq="D")}
)


def _add_calendar_columns(df):
    dt = df["date"].dt

    return df.assign(
        calendar_year="CY " + dt.year.astype(str),
        calendar_year_number=dt.year,
        month=dt.strftime("%b"),
        month_number=dt.month,
        year_month=dt.strftime("%b %Y"),
        year_month_number=dt.year * 12 + dt.month - 1,
        weekday_number=dt.isocalendar().day,
        weekday=dt.strftime("%a"),
        week_number=dt.isocalendar().week,
        start_of_week=dt.to_period("W").dt.start_time,
        end_of_week=lambda x: x["start_of_week"] + pd.DateOffset(days=6),
        start_of_month=dt.to_period("M").dt.start_time,
        end_of_month=dt.to_period("M").dt.end_time.dt.floor("D"),
        same_day_last_year=df["date"] - pd.DateOffset(days=364),
        fiscal_year=dt.to_period("Q-FEB").dt.qyear,
        test=dt.to_period("Q-FEB"),
    )

df = _add_calendar_columns(df)
display(df.head())

,date,calendar_year,calendar_year_number,month,month_number,year_month,year_month_number,weekday_number,weekday,week_number,start_of_week,end_of_week,start_of_month,end_of_month,same_day_last_year,fiscal_year,test
0,2024-02-01,CY 2024,2024,Feb,2,Feb 2024,24289,4,Thu,5,2024-01-29,2024-02-04,2024-02-01,2024-02-29,2023-02-02,2024,2024Q4
1,2024-02-02,CY 2024,2024,Feb,2,Feb 2024,24289,5,Fri,5,2024-01-29,2024-02-04,2024-02-01,2024-02-29,2023-02-03,2024,2024Q4
2,2024-02-03,CY 2024,2024,Feb,2,Feb 2024,24289,6,Sat,5,2024-01-29,2024-02-04,2024-02-01,2024-02-29,2023-02-04,2024,2024Q4
3,2024-02-04,CY 2024,2024,Feb,2,Feb 2024,24289,7,Sun,5,2024-01-29,2024-02-04,2024-02-01,2024-02-29,2023-02-05,2024,2024Q4
4,2024-02-05,CY 2024,2024,Feb,2,Feb 2024,24289,1,Mon,6,2024-02-05,2024-02-11,2024-02-01,2024-02-29,2023-02-06,2024,2024Q4
